In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import base64
import os
import json
import ast
from bs4 import BeautifulSoup

In [3]:
import base64

def base64url_decode(data):
    data = data.encode('utf-8')
    data += b'=' * (4 - (len(data) % 4))
    return base64.urlsafe_b64decode(data)

def is_html_text(text):
    try:
        soup = BeautifulSoup(text, 'html.parser')
        return len(soup.find_all()) > 0
    except:
        return False

def find_text_part(parts):
    res_string = ""
    for part in parts:
        mimeType = part.get('mimeType')
        if mimeType == 'text/plain':
            data = part.get('body', {}).get('data')
            if data:
                text = base64url_decode(data).decode('utf-8')
                if is_html_text(text):
                    soup = BeautifulSoup(text, 'html.parser')
                    res_string = res_string + " " + soup.get_text(separator=' ')
                else:
                    res_string = res_string + " " + text           
        elif mimeType == 'text/html':
            data = part.get('body', {}).get('data')
            if data:
                html = base64url_decode(data).decode('utf-8')
                soup = BeautifulSoup(html, 'html.parser')
                res_string = res_string + " " + soup.get_text(separator=' ')
        elif mimeType == 'multipart/alternative':
            res_string = res_string + " " + find_text_part(part.get('parts', []))
        elif mimeType == 'multipart/related':
            res_string = res_string + " " + find_text_part(part.get('parts', []))
    return res_string

# Function to parse email bodies
def parse_email_body(payload):
    if payload['mimeType'] == 'text/plain':
        data = payload.get('body', {}).get('data')
        if data:
            return base64url_decode(data).decode('utf-8')
    elif payload['mimeType'] == 'text/html':
        data = payload.get('body', {}).get('data')
        if data:
            html = base64url_decode(data).decode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')
            return soup.get_text(separator=' ')
    elif payload['mimeType'].startswith('multipart/'):
        parts = payload.get('parts', [])
        text = find_text_part(parts)
        if text:
            return text
    return ""

def parse_text(x):
    y = ''.join([i if ord(i) < 128 else ' ' for i in x])
    y = ' '.join(y.strip().split())
    return(y)
    

def get_from(x):
    header_ = x.get("headers")
    return(dict([(elem["name"], elem["value"]) for elem in header_ if elem["name"] in ['Subject', 'From', 'To']]))

In [4]:
df = pd.read_csv("emails_dir/email_10-08-2023-13-05-57_10-08-2023-13-23-35.csv")
df.head()

,id,threadId,labelIds,snippet,payload,sizeEstimate,historyId,internalDate
0,189de4921223c782,189de4921223c782,"['CATEGORY_UPDATES', 'INBOX']",PNR/Booking Ref.: XQRKKP Status Date of Bookin...,"{'partId': '', 'mimeType': 'multipart/alternat...",133340,10991039,1691651479000
1,189de46892b19d42,189de46892b19d42,"['Label_86509504764210579', 'CATEGORY_UPDATES'...","Dear Sunil, Your flight booking through our bo...","{'partId': '', 'mimeType': 'multipart/mixed', ...",395710,10991011,1691651311000
2,189de467e12b5f9d,189de467e12b5f9d,"['Label_86509504764210579', 'Label_68765890314...","Dear Card Member, Thank you for using your HDF...","{'partId': '', 'mimeType': 'multipart/alternat...",15945,10991010,1691651307000
3,189de293a9a424ae,189de293a9a424ae,"['Label_4360772061833180153', 'IMPORTANT', 'CA...",Do large language models understand the world?...,"{'partId': '', 'mimeType': 'multipart/alternat...",144576,10990106,1691649314000
4,189de0e819fc210b,189de0e819fc210b,"['UNREAD', 'Label_2234298167017199963', 'CATEG...",Get insights on career opportunities from our ...,"{'partId': '', 'mimeType': 'multipart/alternat...",95364,10990030,1691647638000


In [5]:
import re
def search_pattern(x, pattern = '(.+?)<'):
    try:
        found = re.search(pattern, x).group(1)
    except AttributeError:
        found = np.nan
    return(found) 

In [7]:
def parse_email_body_with_try(payload):
    try:
        x = parse_text(parse_email_body(payload))
        return(x)
    except:
        return("Unable to Parse")    

In [8]:
df = df[[type(ast.literal_eval(elem)) != int for elem in df["payload"]]].reset_index(drop = True)
df["body"] = df["payload"].apply(lambda x: parse_email_body_with_try(ast.literal_eval(x)))

/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:336: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:424: MarkupResemblesLocatorWarning: "http://stat.ethz.ch/education/semesters/ss2014/atsa/Scriptum_v140523.pdf
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:424: MarkupResemblesLocatorWarning: "https://goo.gl/photos/NLzwpxkDF8qRetcs6
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautif

In [9]:
df["headerInfo"] = df["payload"].apply(lambda x:get_from(ast.literal_eval(x)))
df = df.join(pd.json_normalize(df['headerInfo']))

In [10]:
df["fromParsed"] = df["From"].apply(lambda x:search_pattern(str(x), "<(.+?)>"))
df["fromParsed"] = df["fromParsed"].fillna(df["From"])
df

,id,threadId,labelIds,snippet,payload,sizeEstimate,historyId,internalDate,body,headerInfo,From,Subject,To,fromParsed
0,189de4921223c782,189de4921223c782,"['CATEGORY_UPDATES', 'INBOX']",PNR/Booking Ref.: XQRKKP Status Date of Bookin...,"{'partId': '', 'mimeType': 'multipart/alternat...",133340,10991039,1691651479000,PNR/Booking Ref.: XQRKKP Status Date of Bookin...,{'From': 'IndiGo <reservations@customer.goindi...,IndiGo <reservations@customer.goindigo.in>,Your IndiGo Itinerary - XQRKKP,<aagrawal52@gmail.com>,reservations@customer.goindigo.in
1,189de46892b19d42,189de46892b19d42,"['Label_86509504764210579', 'CATEGORY_UPDATES'...","Dear Sunil, Your flight booking through our bo...","{'partId': '', 'mimeType': 'multipart/mixed', ...",395710,10991011,1691651311000,"smartbuy Flight Dear Sunil, Your flight bookin...",{'From': 'SmartBuy <donotreply@smartbuyoffers....,SmartBuy <donotreply@smartbuyoffers.co>,Your Flight Booking with SmartBuy is Successfu...,aagrawal52@gmail.com,donotreply@smartbuyoffers.co
2,189de467e12b5f9d,189de467e12b5f9d,"['Label_86509504764210579', 'Label_68765890314...","Dear Card Member, Thank you for using your HDF...","{'partId': '', 'mimeType': 'multipart/alternat...",15945,10991010,1691651307000,"HDFC BANK Dear Card Member, Thank you for usin...","{'To': 'AAGRAWAL52@gmail.com', 'Subject': 'Ale...",HDFC Bank InstaAlerts <alerts@hdfcbank.net>,Alert : Update on your HDFC Bank Credit Card,AAGRAWAL52@gmail.com,alerts@hdfcbank.net
3,189de293a9a424ae,189de293a9a424ae,"['Label_4360772061833180153', 'IMPORTANT', 'CA...",Do large language models understand the world?...,"{'partId': '', 'mimeType': 'multipart/alternat...",144576,10990106,1691649314000,Do large language models understand the world?...,"{'From': '""The Batch @ DeepLearning.AI"" <theba...","""The Batch @ DeepLearning.AI"" <thebatch@deeple...","AI Cancer Diagnosis Advances, Chatbots Work th...",aagrawal52@gmail.com,thebatch@deeplearning.ai
4,189de0e819fc210b,189de0e819fc210b,"['UNREAD', 'Label_2234298167017199963', 'CATEG...",Get insights on career opportunities from our ...,"{'partId': '', 'mimeType': 'multipart/alternat...",95364,10990030,1691647638000,"""INSEAD MBA Career Statistics"" A World of Tale...","{'From': 'INSEAD MBA <mba.info@insead.edu>', '...",INSEAD MBA <mba.info@insead.edu>,"Aayush, dive into the MBA Career Stats",aagrawal52@gmail.com,mba.info@insead.edu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,13ee5fadbd24dcdb,13ee5fadbd24dcdb,"['IMPORTANT', 'CATEGORY_PERSONAL']",NaN,"{'partId': '', 'mimeType': 'multipart/related'...",1269698,360449,1369658022000,[image: Inline image 1],"{'Subject': 'IIT-JEE/AIEEE', 'From': '""IIT-JEE...","""IIT-JEE/AIEEE"" <eduwavesacademy3@gmail.com>",IIT-JEE/AIEEE,edu waves <eduwavesacademy3@gmail.com>,eduwavesacademy3@gmail.com
2225,13ecd429e3657c7a,13ecd429e3657c7a,"['IMPORTANT', 'CATEGORY_PERSONAL']",FOR ADMISSIONS &amp; DETAILS CONTACT ICFAI - 9...,"{'partId': '', 'mimeType': 'multipart/alternat...",3505,360431,1369243295000,FOR ADMISSIONS & DETAILS CONTACT ICFAI - 91 - ...,{'Subject': 'Fwd: Delivery Status Notification...,Mohammad Zulkarnain <mohammad.zulkarnain12@gma...,Fwd: Delivery Status Notification (Failure),"fastaccess.cafe@gmail.com, vishalshiv8@gmail.c...",mohammad.zulkarnain12@gmail.com
2226,13eac280e2236284,13eac280e2236284,"['IMPORTANT', 'CATEGORY_UPDATES']","Dear Student, Welcome to Flytech Aviation Acad...","{'partId': '', 'mimeType': 'text/html', 'filen...",4642,4500739,1368687906000,"Dear Student, Welcome to Flytech Aviation Acad...","{'From': '""FLYTECH AVIATION"" <FLYTECH@www.emai...","""FLYTECH AVIATION"" <FLYTECH@www.emailibox.com>",Spot admission,aagrawal52@gmail.com,FLYTECH@www.emailibox.com
2227,13e5c04b5626bdd6,13e5c04b5626bdd6,"['IMPORTANT', 'SENT']",mausa ji sirf abhi 1 week ka hi time table ban...,"{'partId': '', 'mimeType': 'multipart/mixed', ...",17460,10263409,1367343412000,mausa ji sirf abhi 1 week ka hi time table ban...,"{'Subject': 'time table(7 days)', 'Fro

In [11]:
df.to_csv("emails_dir/emails_parsed.csv")